In [1]:
!pip install gensim
!pip install networkx
# !pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install pyg-lib torch-scatter torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-geometric
!pip install wandb -qU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 69.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-no

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
# dir = 'gdrive/My Drive/ecm'
dir = 'gdrive/My Drive'
sys.path.append(dir)

Mounted at /content/gdrive


## 1. Import package

In [3]:
import numpy as np
import wandb
import matplotlib.pyplot as plt
import networkx as nx
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, GCNConv, GATConv, Linear, to_hetero, FastRGCNConv, RGCNConv, SAGEConv, GINConv, global_add_pool
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import NeighborSampler
from torch_geometric.utils import remove_self_loops, degree
import copy
import pdb

In [ ]:
wandb.login()
# aa4bc631f2a8ea08a7ea9c077eacdc1d421a7856

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## 2. Function definitions

#### 2.1 Graph reader

In [4]:
def read_graph_pygeo(node_file_name, edge_file_name, n_subarea):
    x = []
    nodefile = open(node_file_name)
    newnode = nodefile.readline()
    while newnode:
        nodeId = int(newnode.split('\t')[0])
        nodeWeight = list()
        for i in range(0, n_subarea):
            nodeWeight.append(float(newnode.split('\t')[i+1]))
        x.append(nodeWeight)
        newnode = nodefile.readline()
    nodefile.close()
    x = torch.tensor(x).type(torch.float)

    edge_index = [[],[]]
    edge_attr = []
    edgefile = open(edge_file_name)
    newedge = edgefile.readline()
    while newedge:
        node1 = int(newedge.split('\t')[0])
        node2 = int(newedge.split('\t')[1])
        edgeWeight = float(newedge.split('\t')[2])
        edge_index[0].append(node1)
        edge_index[1].append(node2)
        edge_attr.append([edgeWeight])
        newedge = edgefile.readline()
    edgefile.close()
    edge_index = torch.tensor(edge_index).type(torch.long)
    edge_attr = torch.tensor(edge_attr)
    
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data


In [5]:
def read_graph_pygeo2(node_file_name, edge_file_name, n_subarea, suba_file_name):
    data_heter = HeteroData()
    data = read_graph_pygeo(node_file_name, edge_file_name, n_subarea)
    
    # add two new attributes of each node (degree and total weight of all the edges)
    _out, _in = data.edge_index
    deg = degree(_out, data.x.size(0), dtype=data.x.dtype).reshape(-1,1)
    x = torch.cat((data.x, deg), 1)
    x = torch.cat((x, torch.zeros(data.x.size()[0],1)), 1)
    for i in range(len(data.x)):
        x[i][-1] = sum(data.edge_attr[data.edge_index[0] == i])
    # add worker nodes
    data_heter['worker'].x = x
    # add subarea nodes   
    x = []
    subafile = open(suba_file_name)
    newnode = subafile.readline()
    while newnode:
        nodeId = int(newnode.split('\t')[0])
        nodeWeight = list([int(newnode.split('\t')[i]) for i in range(1,4)])
        x.append(nodeWeight)
        newnode = subafile.readline()
    subafile.close()
    data_heter['task'].x = torch.tensor(x).type(torch.float)
    
    # add edges between workers
    data_heter['worker','influence','worker'].edge_index = data.edge_index.type(torch.long)

    data_heter['worker','influence','worker'].edge_attr = data.edge_attr
    # add edges between subarea nodes and worker nodes
    x = torch.argsort(-data.x[0])[:5]
    l = list(np.array(x))
    y = torch.stack(data.x[0][x].split(1))
    x = torch.stack((x,0*torch.ones(x.size()[0])),0)
    for i in range(1,len(data.x)):
        x = torch.cat((x, torch.stack((torch.argsort(-data.x[i])[:5],i*torch.ones(torch.argsort(-data.x[i])[:5].size()[0])),0)),1)
        y = torch.cat((y, torch.stack(data.x[i][torch.argsort(-data.x[i])[:5]].split(1))),0)
        t = np.array(torch.argsort(-data.x[i])[:5])+100*i
        l += list(t)
    data_heter['task','allocated','worker'].edge_index , data_heter['task','allocated','worker'].edge_attr = x.type(torch.long),y
    return data_heter,l


#### 2.2 Tools

In [6]:
def one_time_monte_carlo_simulation_new(task, data, curr_active_node):
    all_activated_node = curr_active_node
    next_active_node = set()
    mc_pairs = []
    
    for node in curr_active_node:
        loader = NeighborSampler(edge_index=data.edge_index, sizes=[-1], node_idx=torch.tensor([node]), batch_size=1)
        Neighbordata = next(iter(loader))
        for i in range(len(Neighbordata[1][1:])):
            nbr = int(Neighbordata[1][i+1])
            randval = random.random()
            if (nbr not in all_activated_node) and randval<float(data.edge_attr[Neighbordata[2].edge_index[0,i]])*data.x[node][task]:
                next_active_node.add(nbr)

    while len(curr_active_node) != 0 and len(next_active_node) != 0:
        curr_active_node = next_active_node.copy()
        all_activated_node = all_activated_node | curr_active_node
        next_active_node = set()
        for node in curr_active_node:
            loader = NeighborSampler(edge_index=data.edge_index, sizes=[-1], node_idx=torch.tensor([node]), batch_size=1)
            Neighbordata = next(iter(loader))
            for i in range(len(Neighbordata[1][1:])):
                nbr = int(Neighbordata[1][i+1])
                randval = random.random()
                if (nbr not in all_activated_node) and randval<float(data.edge_attr[Neighbordata[2].edge_index[0,i]])*data.x[node][task]:
                    next_active_node.add(nbr)

    return all_activated_node


# Get monte carlo probability
def monte_carlo_simulations_new(MC_times, data, curr_node):
    seed_pair = {}
    for i in curr_node:
      t,u = i//n_users, i%n_users
      if t not in seed_pair.keys():
        seed_pair[t] = [u]
      else:
        seed_pair[t].append(u)
    inf = 0
    for task in seed_pair.keys():
      curr_node = set(seed_pair[task])
      inf_single = 0
      for i in range(0, MC_times):
          inf_single += len(one_time_monte_carlo_simulation_new(task, data, curr_node))
      inf += inf_single
    return inf/MC_times

#### 2.3 Influential GAT for Social Graph

In [38]:
class GAT_social(nn.Module):
    def __init__(self,  ofeat):
        super().__init__()
        # parameters for the GAT
        self.conv1 = GATConv((-1, -1), ofeat, add_self_loops=False)
        self.conv2 = GATConv((-1, -1), ofeat, add_self_loops=False)
        # for the second layer
        self.conv3 = GATConv((-1, -1), ofeat, add_self_loops=False)
        self.conv4 = GATConv((-1, -1), ofeat, add_self_loops=False)
        # parameters for the linear layer
        self.W3 = Linear(-1, ofeat, bias = True)
        self.W3.weight.data.normal_(0, 0.1)
        self.W4 = Linear(-1, ofeat, bias = True)
        self.W4.weight.data.normal_(0, 0.1)
        self.W5 = Linear(-1, ofeat, bias = True)
        self.W5.weight.data.normal_(0, 0.1)
        self.W6 = Linear(-1, ofeat, bias = True)
        self.W6.weight.data.normal_(0, 0.1)

    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x1 = self.conv1(x, edge_index)
      x1 = F.relu(x1)
      edge_index_rev = torch.flip(edge_index, dims=[0])
      x2 = self.conv2(x, edge_index_rev)
      x2 = F.relu(x2)
      # for the second layer
      # x1 = self.conv3(x1, edge_index)
      # x1 = F.relu(x1)
      # x2 = self.conv4(x2, edge_index_rev)
      # x2 = F.relu(x2)
      G1 = torch.sigmoid(self.W3(x1)+self.W4(x2))
      G2 = torch.sigmoid(self.W5(x1)+self.W6(x2))
      # element-wise multiplication of G1 and x1, G2 and x2, then concatenate them
      return torch.cat((torch.mul(G1, x1), torch.mul(G2, x2)), dim = 1)    
      

In [39]:
class GAT_social_2(nn.Module):
    def __init__(self,  ofeat):
        super().__init__()
        # parameters for the GAT
        self.conv1 = GATConv((-1, -1), ofeat, add_self_loops=False)
        self.conv2 = GATConv((-1, -1), ofeat, add_self_loops=False)
        # for the second layer
        self.conv3 = GATConv((-1, -1), ofeat, add_self_loops=False)
        self.conv4 = GATConv((-1, -1), ofeat, add_self_loops=False)
        # parameters for the linear layer
        self.W3 = Linear(-1, ofeat, bias = True)
        self.W3.weight.data.normal_(0, 0.1)
        self.W4 = Linear(-1, ofeat, bias = True)
        self.W4.weight.data.normal_(0, 0.1)
        self.W5 = Linear(-1, ofeat, bias = True)
        self.W5.weight.data.normal_(0, 0.1)
        self.W6 = Linear(-1, ofeat, bias = True)
        self.W6.weight.data.normal_(0, 0.1)

    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x1 = self.conv1(x, edge_index)
      x1 = F.relu(x1)
      edge_index_rev = torch.flip(edge_index, dims=[0])
      x2 = self.conv2(x, edge_index_rev)
      x2 = F.relu(x2)
      # for the second layer
      x1 = self.conv3(x1, edge_index)
      x1 = F.relu(x1)
      x2 = self.conv4(x2, edge_index_rev)
      x2 = F.relu(x2)
      G1 = torch.sigmoid(self.W3(x1)+self.W4(x2))
      G2 = torch.sigmoid(self.W5(x1)+self.W6(x2))
      # element-wise multiplication of G1 and x1, G2 and x2, then concatenate them
      return torch.cat((torch.mul(G1, x1), torch.mul(G2, x2)), dim = 1)    
      

In [8]:
# comparation with normal GAT
class GAT_normal(nn.Module):
    def __init__(self,  ofeat):
        super().__init__()
        # parameters for the GAT
        self.conv1 = GATConv((-1, -1), ofeat, add_self_loops=False)
        self.conv2 = GATConv((-1, -1), ofeat, add_self_loops=False)
        # parameters for the linear layer
        self.W3 = Linear(-1, ofeat, bias = True)
        self.W3.weight.data.normal_(0, 0.1)
       
    def forward(self, data):
      x, edge_index = data.x, data.edge_index
      x1 = self.conv1(x, edge_index)
      x1 = F.relu(x1)
      return torch.sigmoid(self.W3(x1))
      

In [9]:
# comparation with GraphSAGE
class GraphSAGE(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv((-1,-1), hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.W3 = Linear(-1, out_channels, bias = True)
        self.W3.weight.data.normal_(0, 0.1)
        
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return torch.sigmoid(self.W3(x))

In [10]:
class GINNet(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes):
        super(GINNet, self).__init__()

        # GIN layers
        self.conv1 = GINConv(torch.nn.Sequential(
            torch.nn.Linear(num_features, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim)
        ))
        self.conv2 = GINConv(torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim)
        ))

        # MLP classifier
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GIN layer
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))

        # Global pooling
        x = global_add_pool(x, batch)

        # MLP classifier
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

#### 2.4 Relational GCN and Correlation Layer for Worker-Task Graph

In [11]:
class RGCN(nn.Module):
    def __init__(self, out_channels):
        super(RGCN, self).__init__()
        self.conv = FastRGCNConv(2*out_channels, out_channels,
                              num_relations=num_relations)
        self.lins = torch.nn.ModuleList()
        for i in range(len(node_types)):
            lin = Linear(-1, 2*out_channels)
            lin.weight.data.normal_(0, 0.1)
            self.lins.append(lin)

    def trans_dimensions(self, g):
        data = copy.deepcopy(g)
        for node_type, lin in zip(node_types, self.lins):
            data[node_type].x = lin(data[node_type].x.float())
        return data

    def forward(self, data):
        data = self.trans_dimensions(data)
        homogeneous_data = data.to_homogeneous()
        edge_index, edge_type = homogeneous_data.edge_index, homogeneous_data.edge_type
        x = self.conv(homogeneous_data.x, edge_index, edge_type)
        return x.relu()

##### 2.5 Deep Q Network

In [12]:
# aggregate the features of the nodes in the same class
class state_aggregation_layer(nn.Module):
    def __init__(self,ifeat1, ifeat2, ifeat3):
        super().__init__()
        self.ifeat1 = ifeat1
        self.ifeat2 = ifeat2
        self.ifeat3 = ifeat3
        # parameters for the linear layer
        self.W11 = Linear(-1, self.ifeat1, bias = True)
        self.W11.weight.data.normal_(0, 0.1)
        # self.W12 = Linear(-1, self.ifeat1, bias = True)
        # self.W13 = Linear(-1, self.ifeat1, bias = True)
        # self.W21 = Linear(-1, self.ifeat2, bias = True)
        # self.W22 = Linear(-1, self.ifeat2, bias = True)
        # self.W23 = Linear(-1, self.ifeat2, bias = True)
        # self.W31 = Linear(-1, self.ifeat3, bias = True)
        # self.W32 = Linear(-1, self.ifeat3, bias = True)
        # self.W33 = Linear(-1, self.ifeat3, bias = True)
        
    def forward(self, x_t_rc, x_v_rc, x_v_s):
      # G1 = torch.sigmoid(self.W11(x_t_rc)+self.W12(x_v_rc)+self.W13(x_v_s))
      # G2 = torch.sigmoid(self.W21(x_t_rc)+self.W22(x_v_rc)+self.W23(x_v_s))
      # G3 = torch.sigmoid(self.W31(x_t_rc)+self.W32(x_v_rc)+self.W33(x_v_s))
      # tmp = torch.cat((torch.mul(G1,x_t_rc),torch.mul(G2,x_v_rc))) 
      # return torch.cat((tmp,torch.mul(G3,x_v_s)))
      tmp = torch.cat((x_t_rc,x_v_rc)) 
      tmp = torch.cat((tmp,x_v_s))
      G1 = torch.sigmoid(self.W11(tmp))
      return G1

In [13]:
class Net(nn.Module):
    def __init__(self):                                               
        super(Net, self).__init__()     
        self.fc1 = Linear(-1, 50)                                    
        self.fc1.weight.data.normal_(0, 0.1)
        self.fc2 = Linear(50, 20)                                    
        self.fc2.weight.data.normal_(0, 0.1)                                     
        self.fc3 = Linear(20, N_ACTIONS)                                     
        self.fc3.weight.data.normal_(0, 0.1)
        
    def forward(self, x):                                                  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        # x = self.fc3(x)
        # x = (x - x.min())/(x.max()-x.min())
        return x

class Env():
    def __init__(self, data1, data2, k): 
        self.reset()
        self.data1 = data1
        self.data2 = data2
        self.k = k
        self.steps = 0
    
    def reset(self):
        self.all_activated_node = set()
        # self.state = np.zeros(2*work_cha+task_cha+agg_cha)
        self.state = np.zeros(2*work_cha)
        self.currnode = set()
        self.steps = 0
        return self.state
    
    def step(self, action, gat_net, gcn_net, agg_net):
        reward = monte_carlo_simulations_new(MC_times,  self.data1, self.currnode)
        self.currnode.add(action)
        reward = monte_carlo_simulations_new(MC_times,  self.data1, self.currnode) - reward
        self.steps += 1
        n = gcn_net(self.data2)
        t = gat_net(self.data1)[action%n_users]
        self.state = self.state + agg_net(t, n[action%n_users], n[action//n_users]).cpu().detach().numpy()
        done = (self.steps == self.k)
        return self.state, reward, done

            
class DQN(nn.Module):
    def __init__(self):    
        super(DQN, self).__init__()                                                       
        self.eval_net, self.target_net = Net(), Net()
        # for param in self.target_net.features.parameters():
        #   param.requires_grad = False
        self.target_net.fc1.weight.requires_grad = False
        self.target_net.fc2.weight.requires_grad = False
        self.target_net.fc3.weight.requires_grad = False
        self.gcn_net = RGCN(task_cha)
        self.gat_net = GAT_social(work_cha)
        self.agg_net = state_aggregation_layer(2*work_cha,task_cha,agg_cha)                          
        self.learn_step_counter = 0                                             # for target updating
        self.memory_counter = 0                                                 # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))   
        self.eps = EPSILON
        self.optimizer = torch.optim.Adam(self.parameters(), lr=LR)
        # self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss = nn.MSELoss()

    def choose_action(self, x, chosen_index_set, left_index_set):                                                                     
        if np.random.uniform() < self.eps :
            x = torch.tensor(x, dtype=torch.float).to(device) 
            x = self.eval_net(x).cpu().detach() 
            if len(chosen_index_set):
              x = x.index_fill(0,torch.tensor(list(chosen_index_set)),-float('inf'))                      
            action = x.argmax().item()
        else:                                                             
            action = random.choice(list(left_index_set))
        return action    

    def choose_action_ran(self, x, chosen_index_set, left_index_set):                                                                     
        x = torch.tensor(x, dtype=torch.float).to(device) 
        x = self.eval_net(x).cpu().detach() 
        if len(chosen_index_set):
          x = x.index_fill(1,torch.tensor(list(chosen_index_set)),-float('inf'))                      
        action = x.argmax().item()
        return action                                                       

    def store_transition(self, s, a, r, s_):                                    
        transition = np.hstack((s, a, r, s_))              
        index = self.memory_counter % MEMORY_CAPACITY                         
        self.memory[index, :] = transition                                      
        self.memory_counter += 1                                               

    def learn(self):                                       
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:                 
            self.target_net.load_state_dict(self.eval_net.state_dict())        
        self.learn_step_counter += 1                                         

        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)           
        b_memory = self.memory[sample_index, :]                                 
        b_s = torch.FloatTensor(b_memory[:, :N_STATES]).to(device)
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)).to(device)
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]).to(device)
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:]).to(device)

        q_eval = self.eval_net(b_s).gather(1, b_a)
        q_next = self.target_net(b_s_).detach()
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)
        # pdb.set_trace()

        l = self.loss(q_eval, q_target)
        self.optimizer.zero_grad()                                     
        l.backward()                                               
        self.optimizer.step()   
        # wandb.log({'Loss': l,'Q_eval_max':q_eval.max().cpu().detach().numpy(),'Q_eval_mean':q_eval.mean().cpu().detach().numpy(),'Q_targ_max':q_target.max().cpu().detach().numpy(),'Q_target_mean':q_target.mean().cpu().detach().numpy()}) 

        return 

## 3. Experiments

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
n_subarea = 100
n_users = 3000
input_file_path = dir+'/New_Gowalla'
input_index = 0
nodefilename = input_file_path + '/input_node_' + str(n_users) + '_' + str(input_index) + '.txt'
edgefilename = input_file_path + '/input_edge_' + str(n_users) + '_' + str(input_index) + '.txt'
subafilename = input_file_path + '/subarea_pos_subarea_' + str(n_subarea) + '_node_' + str(n_users) + '_' + str(input_index) + '.txt'

data = read_graph_pygeo(nodefilename, edgefilename, n_subarea).to(device)
data2,l = read_graph_pygeo2(nodefilename, edgefilename, n_subarea, subafilename)
data2.to(device)

HeteroData(
  worker={ x=[3000, 102] },
  task={ x=[100, 3] },
  (worker, influence, worker)={
    edge_index=[2, 3350],
    edge_attr=[3350, 1]
  },
  (task, allocated, worker)={
    edge_index=[2, 15000],
    edge_attr=[15000, 1]
  }
)

In [41]:
# GNN parameters
work_cha = 5
task_cha = 10
agg_cha = 10
# gat_net = GAT_social(work_cha)
gat_net = GAT_social_2(work_cha)
# gat_net = GAT_normal(work_cha)
# gat_net = GraphSAGE(1,work_cha)
node_types, edge_types = data2.metadata()
num_relations = len(edge_types)
gcn_net = RGCN(task_cha)
agg_net = state_aggregation_layer(2*work_cha,task_cha,agg_cha)

In [42]:
# DRL parameters
BATCH_SIZE = 32                               
LR = 0.001                                      
EPSILON = 0.9                                   
GAMMA = 0.9                                    
TARGET_REPLACE_ITER = 200                    
MEMORY_CAPACITY = 100                    
N_ACTIONS = n_users * n_subarea
# N_STATES = 2*work_cha+task_cha+agg_cha
N_STATES = 2*work_cha
MC_times = 25
K = 100
EPOCH = 40

In [43]:
def train(n_sample_G, path=None):
    print("n user: %d"%n_users)
    print("n seed: %d"%K)
    # wandb.init(config={"nuser":n_users,"nseed":K,"Com":"light weight"},project="ECM-final")
    
    dqn = DQN()
    if path:
      dqn.load_state_dict(torch.load(path, map_location=device))  

    for input_index in range(n_sample_G):
        nodefilename = input_file_path + '/input_node_' + str(n_users) + '_' + str(input_index) + '.txt'
        edgefilename = input_file_path + '/input_edge_' + str(n_users) + '_' + str(input_index) + '.txt'
        subafilename = input_file_path + '/subarea_pos_subarea_' + str(n_subarea) + '_node_' + str(n_users) + '_' + str(input_index) + '.txt'
        data = read_graph_pygeo(nodefilename, edgefilename, n_subarea).to(device)
        data2,l = read_graph_pygeo2(nodefilename, edgefilename, n_subarea, subafilename)
        data2.to(device)                                                        
        env = Env(data,data2,K)


        dqn.train()
        dqn.to(device)

        max_reward_sum = 0


        for i in range(EPOCH):                                                 
            print('<<<<<<<<<Episode: %s' % i)
            s = env.reset()                                  
            episode_reward_sum = 0    
            all_action = set(l)                                    

            while True:                                                        
                a = dqn.choose_action(s,env.currnode,all_action - env.currnode)                             
                s_, r, done = env.step(a, dqn.gat_net, dqn.gcn_net, dqn.agg_net)                         
                dqn.store_transition(s, a, r, s_)                 
                episode_reward_sum += r                          
                s = s_          
                # wandb.log({'steps':env.steps ,  'reward':r, 'action':a})                                      

                if dqn.memory_counter > MEMORY_CAPACITY:              
                    dqn.learn()
                    

                if done:  
                    dqn.eps = (1+dqn.eps)/2
                    # wandb.log({'epoch': i,  'reward_sum':round(episode_reward_sum, 2)})   
                    print('episode%s---reward_sum: %s' % (i, round(episode_reward_sum, 2)))
                    break
            EC = monte_carlo_simulations_new(MC_times,  data, env.currnode)
            print('episode%s---reward_EC: %s' % (i, round(EC, 2)))
                    
            if episode_reward_sum > max_reward_sum:
                max_reward_sum = episode_reward_sum
                PATH = input_file_path + '/model_' + str(n_users) + '_' + str(K) + '_' + str(round(episode_reward_sum, 2)) + '.txt'
                torch.save(dqn.state_dict(), PATH) 
            

In [44]:
#Train
input_index = 0
n_subarea = 100
n_users = 3000 # 5000
K = 25 # 50,75,100
N_ACTIONS = n_users * n_subarea
n_sample = 1 # sample n networks to generalize the model 
train(n_sample,0)

n user: 3000
n seed: 25
<<<<<<<<<Episode: 0
episode0---reward_sum: 24.8
episode0---reward_EC: 25.08
<<<<<<<<<Episode: 1
episode1---reward_sum: 25.12
episode1---reward_EC: 25.04
<<<<<<<<<Episode: 2
episode2---reward_sum: 25.2
episode2---reward_EC: 25.04
<<<<<<<<<Episode: 3
episode3---reward_sum: 25.2
episode3---reward_EC: 25.08
<<<<<<<<<Episode: 4
episode4---reward_sum: 25.24
episode4---reward_EC: 25.12
<<<<<<<<<Episode: 5
episode5---reward_sum: 25.16
episode5---reward_EC: 25.12
<<<<<<<<<Episode: 6
episode6---reward_sum: 25.44
episode6---reward_EC: 25.08
<<<<<<<<<Episode: 7
episode7---reward_sum: 25.96
episode7---reward_EC: 25.16
<<<<<<<<<Episode: 8
episode8---reward_sum: 24.52
episode8---reward_EC: 25.04
<<<<<<<<<Episode: 9
episode9---reward_sum: 24.72
episode9---reward_EC: 25.0
<<<<<<<<<Episode: 10
episode10---reward_sum: 24.76
episode10---reward_EC: 25.08
<<<<<<<<<Episode: 11
episode11---reward_sum: 25.2
episode11---reward_EC: 25.2
<<<<<<<<<Episode: 12
episode12---reward_sum: 24.8
ep

In [27]:
BATCH_SIZE = 1
def test(path):
  print("n user: %d"%n_users)
  print("n seed: %d"%K)

  dqn = DQN()   
  dqn.load_state_dict(torch.load(path, map_location=device))  
  dqn.to(device) 
  dqn.eval()

  n = dqn.gcn_net(data2)
  t = dqn.gat_net(data)
  k = dqn.agg_net

  env = Env(data,data2,K)
  s = env.reset()                                  
  episode_reward_sum = 0
  done = 0    
  all_action = set(l)  
  curr = set()                                       
  A=time.time() 
  for i in range(K):                                                          
    a = dqn.choose_action_ran(s,env.currnode,all_action-env.currnode)
    # print(a)                  
    s = env.step_ran(a, dqn.gat_net, dqn.gcn_net, dqn.agg_net)   
  B=time.time() 
  EC = monte_carlo_simulations_new(MC_times,  data, env.currnode)
  print('---reward_EC: %s' % ( round(EC, 2)))
  print('time ',B-A)

In [32]:
# Test on the test dataset
n_subarea = 100
n_users = 3000
input_file_path = dir+'/New_Gowalla'
input_index = 1
nodefilename = input_file_path + '/input_node_' + str(n_users) + '_' + str(input_index) + '.txt'
edgefilename = input_file_path + '/input_edge_' + str(n_users) + '_' + str(input_index) + '.txt'
subafilename = input_file_path + '/subarea_pos_subarea_' + str(n_subarea) + '_node_' + str(n_users) + '_' + str(input_index) + '.txt'

data = read_graph_pygeo(nodefilename, edgefilename, n_subarea).to(device)
data2,l = read_graph_pygeo2(nodefilename, edgefilename, n_subarea, subafilename)
data2.to(device)

PATH = input_file_path + '/model_' + str(n_users) + '_' + str(25) + '_' + str(25.76) + '.txt'
test(PATH)

n user: 3000
n seed: 25


AttributeError: ignored